# BSP Floor Plan Generator

Clean BSP algorithm for generating floor plans.

Output: 2D numpy array where `0 = floor/empty`, `1 = wall`, `2 = door`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import random
from typing import List, Tuple, Optional
from dataclasses import dataclass

In [ ]:
def display_floor_plan(grid: np.ndarray, title: str = "Floor Plan", figsize: Tuple[int, int] = (10, 10)):
    """Display a floor plan with a nice color scheme."""
    fig, ax = plt.subplots(figsize=figsize)
    # 0 = floor (white), 1 = wall (dark), 2 = door (brown)
    cmap = ListedColormap(['#FFFFFF', '#2C3E50', '#8B4513'])
    ax.imshow(grid, cmap=cmap, interpolation='nearest', vmin=0, vmax=2)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticks(np.arange(-0.5, grid.shape[1], 1), minor=True)
    ax.set_yticks(np.arange(-0.5, grid.shape[0], 1), minor=True)
    ax.grid(which='minor', color='#BDC3C7', linewidth=0.5, alpha=0.5)
    plt.tight_layout()
    plt.show()

def display_multiple(grids: List[np.ndarray], titles: List[str], cols: int = 3, figsize: Tuple[int, int] = (15, 5)):
    """Display multiple floor plans side by side."""
    rows = (len(grids) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(figsize[0], figsize[1] * rows))
    axes = np.atleast_2d(axes).flatten()
    # 0 = floor (white), 1 = wall (dark), 2 = door (brown)
    cmap = ListedColormap(['#FFFFFF', '#2C3E50', '#8B4513'])
    for idx, (grid, title) in enumerate(zip(grids, titles)):
        axes[idx].imshow(grid, cmap=cmap, interpolation='nearest', vmin=0, vmax=2)
        axes[idx].set_title(title, fontsize=11, fontweight='bold')
        axes[idx].set_xticks([])
        axes[idx].set_yticks([])
    for idx in range(len(grids), len(axes)):
        axes[idx].axis('off')
    plt.tight_layout()
    plt.show()

---
## BSP Algorithm

In [ ]:
@dataclass
class BSPNode:
    """A node in the BSP tree representing a rectangular region."""
    x: int
    y: int
    width: int
    height: int
    left: Optional['BSPNode'] = None
    right: Optional['BSPNode'] = None
    room: Optional[Tuple[int, int, int, int]] = None

class BSPFloorPlan:
    """Generate floor plans using Binary Space Partitioning."""
    
    SHAPES = ['rectangle', 'trapezoid', 'h_shape', 't_shape', 'pentagon', 
              'oval', 'u_shape', 'parallelogram', 'semicircle', 'triangle']
    
    def __init__(
        self,
        width: int = 50,
        height: int = 50,
        min_room_size: int = 6,
        max_room_size: int = 15,
        wall_thickness: int = 1,
        door_width: int = 2,
        split_variance: float = 0.3,
        building_shape: str = 'rectangle'
    ):
        self.width = width
        self.height = height
        self.min_room_size = min_room_size
        self.max_room_size = max_room_size
        self.wall_thickness = wall_thickness
        self.door_width = door_width
        self.split_variance = split_variance
        self.building_shape = building_shape.lower()
        
    def generate(self) -> np.ndarray:
        """Generate a new floor plan."""
        # Create shape mask FIRST
        self.mask = self._create_shape_mask()
        
        # Create interior mask (eroded by wall_thickness)
        self.interior = self._erode_mask(self.mask, self.wall_thickness)
        
        # Start with all empty (0) - outside is empty
        self.grid = np.zeros((self.height, self.width), dtype=np.int8)
        
        # Draw the outer wall (mask minus interior)
        outer_wall = self.mask & ~self.interior
        self.grid[outer_wall] = 1
        
        # Create BSP tree
        root = BSPNode(0, 0, self.width, self.height)
        self._split(root)
        
        # Add internal walls between rooms
        self._add_internal_walls(root)
        
        # Connect rooms with doors
        self._connect_rooms(root)
        
        return self.grid
    
    def _erode_mask(self, mask: np.ndarray, iterations: int) -> np.ndarray:
        """Erode a mask by a number of pixels."""
        result = mask.copy()
        for _ in range(iterations):
            new_result = np.zeros_like(result)
            for y in range(1, self.height - 1):
                for x in range(1, self.width - 1):
                    if result[y, x]:
                        # Only keep if all 4 neighbors are also in mask
                        if result[y-1, x] and result[y+1, x] and result[y, x-1] and result[y, x+1]:
                            new_result[y, x] = True
            result = new_result
        return result
    
    def _create_shape_mask(self) -> np.ndarray:
        """Create a boolean mask for the building shape. True = inside building."""
        mask = np.zeros((self.height, self.width), dtype=bool)
        h, w = self.height, self.width
        
        if self.building_shape == 'rectangle':
            mask[:, :] = True
            
        elif self.building_shape == 'trapezoid':
            for y in range(h):
                t = y / max(1, h - 1)
                indent = int((1 - t) * w * 0.25)
                mask[y, indent:w-indent] = True
                
        elif self.building_shape == 'h_shape':
            bar_width = w // 4
            mask[:, :bar_width] = True
            mask[:, w-bar_width:] = True
            mid_start = h // 3
            mid_end = 2 * h // 3
            mask[mid_start:mid_end, :] = True
            
        elif self.building_shape == 't_shape':
            top_height = h // 3
            stem_width = w // 3
            stem_start = (w - stem_width) // 2
            mask[:top_height, :] = True
            mask[top_height:, stem_start:stem_start+stem_width] = True
            
        elif self.building_shape == 'pentagon':
            cx = w // 2
            for y in range(h):
                for x in range(w):
                    if y > h * 0.3:
                        if abs(x - cx) < w * 0.45:
                            mask[y, x] = True
                    else:
                        slope = (w * 0.45) / (h * 0.3)
                        allowed_x = slope * y
                        if abs(x - cx) < allowed_x:
                            mask[y, x] = True
                            
        elif self.building_shape == 'oval':
            cx, cy = w // 2, h // 2
            rx, ry = w // 2 - 1, h // 2 - 1
            for y in range(h):
                for x in range(w):
                    if ((x - cx) / max(1, rx)) ** 2 + ((y - cy) / max(1, ry)) ** 2 <= 1:
                        mask[y, x] = True
                        
        elif self.building_shape == 'u_shape':
            bar_width = w // 3
            bottom_height = h // 4
            mask[h-bottom_height:, :] = True
            mask[:, :bar_width] = True
            mask[:, w-bar_width:] = True
            
        elif self.building_shape == 'parallelogram':
            skew = w // 4
            for y in range(h):
                t = y / max(1, h - 1)
                offset = int(t * skew)
                x_start = offset
                x_end = w - skew + offset
                if x_start < w and x_end > 0:
                    mask[y, max(0, x_start):min(w, x_end)] = True
                    
        elif self.building_shape == 'semicircle':
            cx = w // 2
            r = min(w // 2, h) - 1
            for y in range(h):
                for x in range(w):
                    dist = ((x - cx) ** 2 + (y - 0) ** 2) ** 0.5
                    if dist <= r:
                        mask[y, x] = True
                        
        elif self.building_shape == 'triangle':
            cx = w // 2
            for y in range(h):
                t = y / max(1, h - 1)
                half_width = int(t * w / 2)
                if half_width > 0:
                    mask[y, cx-half_width:cx+half_width] = True
                elif y == 0:
                    mask[y, cx] = True
        else:
            mask[:, :] = True
            
        return mask
    
    def _region_has_space(self, x: int, y: int, w: int, h: int) -> bool:
        """Check if a rectangular region has enough space inside the interior."""
        if x < 0 or y < 0 or x + w > self.width or y + h > self.height:
            return False
        region = self.interior[y:y+h, x:x+w]
        return np.sum(region) >= (w * h * 0.3)
    
    def _split(self, node: BSPNode, depth: int = 0) -> None:
        """Recursively split a node into two children."""
        if node.width < self.min_room_size * 2 or node.height < self.min_room_size * 2:
            return
        if max(node.width, node.height) < self.max_room_size and random.random() < 0.3:
            return

        if node.width > node.height * 1.25:
            horizontal = False
        elif node.height > node.width * 1.25:
            horizontal = True
        else:
            horizontal = random.random() < 0.5

        if horizontal:
            max_split = node.height - self.min_room_size
            min_split = self.min_room_size
            if max_split <= min_split:
                return
            variance = int((max_split - min_split) * self.split_variance)
            mid = node.height // 2
            split = mid + random.randint(-variance, variance)
            split = max(min_split, min(max_split, split))
            
            left = BSPNode(node.x, node.y, node.width, split)
            right = BSPNode(node.x, node.y + split, node.width, node.height - split)
            
            if self._region_has_space(left.x, left.y, left.width, left.height):
                node.left = left
            if self._region_has_space(right.x, right.y, right.width, right.height):
                node.right = right
        else:
            max_split = node.width - self.min_room_size
            min_split = self.min_room_size
            if max_split <= min_split:
                return
            variance = int((max_split - min_split) * self.split_variance)
            mid = node.width // 2
            split = mid + random.randint(-variance, variance)
            split = max(min_split, min(max_split, split))
            
            left = BSPNode(node.x, node.y, split, node.height)
            right = BSPNode(node.x + split, node.y, node.width - split, node.height)
            
            if self._region_has_space(left.x, left.y, left.width, left.height):
                node.left = left
            if self._region_has_space(right.x, right.y, right.width, right.height):
                node.right = right

        if node.left:
            self._split(node.left, depth + 1)
        if node.right:
            self._split(node.right, depth + 1)
    
    def _add_internal_walls(self, node: BSPNode) -> None:
        """Add thin internal walls at BSP split boundaries."""
        if node.left is None and node.right is None:
            node.room = (node.x, node.y, node.width, node.height)
            return
            
        if node.left:
            self._add_internal_walls(node.left)
        if node.right:
            self._add_internal_walls(node.right)
        
        if node.left and node.right:
            if node.left.y == node.right.y:
                # Vertical split - draw vertical wall
                wall_x = node.left.x + node.left.width
                for y in range(node.y, node.y + node.height):
                    if 0 <= y < self.height and 0 <= wall_x < self.width:
                        if self.mask[y, wall_x]:
                            self.grid[y, wall_x] = 1
            else:
                # Horizontal split - draw horizontal wall
                wall_y = node.left.y + node.left.height
                for x in range(node.x, node.x + node.width):
                    if 0 <= wall_y < self.height and 0 <= x < self.width:
                        if self.mask[wall_y, x]:
                            self.grid[wall_y, x] = 1
    
    def _get_room(self, node: BSPNode) -> Optional[Tuple[int, int, int, int]]:
        """Get a room from this node or its descendants."""
        if node.room:
            return node.room
        
        left_room = self._get_room(node.left) if node.left else None
        right_room = self._get_room(node.right) if node.right else None
        
        if left_room and right_room:
            return random.choice([left_room, right_room])
        return left_room or right_room
    
    def _connect_rooms(self, node: BSPNode) -> None:
        """Create doorways between adjacent rooms. Doors marked as 2."""
        if node.left is None or node.right is None:
            return
        
        self._connect_rooms(node.left)
        self._connect_rooms(node.right)
        
        if node.left.y == node.right.y:
            # Vertical wall
            wall_x = node.left.x + node.left.width
            valid_ys = []
            for y in range(node.y + 1, node.y + node.height - 1):
                if 0 <= y < self.height and 0 <= wall_x < self.width:
                    if self.interior[y, wall_x]:
                        valid_ys.append(y)
            if valid_ys:
                mid = len(valid_ys) // 2
                door_y = valid_ys[mid]
                for dy in range(self.door_width):
                    py = door_y + dy
                    if 0 <= py < self.height:
                        self.grid[py, wall_x] = 2  # Door
        else:
            # Horizontal wall
            wall_y = node.left.y + node.left.height
            valid_xs = []
            for x in range(node.x + 1, node.x + node.width - 1):
                if 0 <= wall_y < self.height and 0 <= x < self.width:
                    if self.interior[wall_y, x]:
                        valid_xs.append(x)
            if valid_xs:
                mid = len(valid_xs) // 2
                door_x = valid_xs[mid]
                for dx in range(self.door_width):
                    px = door_x + dx
                    if 0 <= px < self.width:
                        self.grid[wall_y, px] = 2  # Door

In [ ]:
# Demo: BSP Floor Plan with oval shape
bsp = BSPFloorPlan(width=50, height=50, min_room_size=8, split_variance=0.4, 
                   wall_thickness=2, building_shape='oval')
plan = bsp.generate()
display_floor_plan(plan, "BSP Floor Plan - Oval")

In [ ]:
# Gallery of variations
plans = []
titles = []

for i in range(6):
    bsp = BSPFloorPlan(
        width=50, 
        height=50, 
        min_room_size=7,
        split_variance=0.4
    )
    plans.append(bsp.generate())
    titles.append(f"Plan {i+1}")

display_multiple(plans, titles, cols=3, figsize=(15, 10))

In [ ]:
# Gallery of all building shapes
plans = []
titles = []

for shape in BSPFloorPlan.SHAPES:
    bsp = BSPFloorPlan(
        width=50, 
        height=50, 
        min_room_size=6,
        split_variance=0.4,
        wall_thickness=2,
        building_shape=shape
    )
    plans.append(bsp.generate())
    titles.append(shape.replace('_', ' ').title())

display_multiple(plans, titles, cols=5, figsize=(20, 8))

---
## Playground

In [ ]:
# === CUSTOMIZE ===

WIDTH = 60
HEIGHT = 60
MIN_ROOM_SIZE = 8
WALL_THICKNESS = 2  # Outer wall thickness
SPLIT_VARIANCE = 0.4
# Options: 'rectangle', 'trapezoid', 'h_shape', 't_shape', 'pentagon',
#          'oval', 'u_shape', 'parallelogram', 'semicircle', 'triangle'
BUILDING_SHAPE = 'oval'

bsp = BSPFloorPlan(
    width=WIDTH,
    height=HEIGHT,
    min_room_size=MIN_ROOM_SIZE,
    wall_thickness=WALL_THICKNESS,
    split_variance=SPLIT_VARIANCE,
    building_shape=BUILDING_SHAPE,
)
plan = bsp.generate()
display_floor_plan(plan, f"BSP Floor Plan - {BUILDING_SHAPE.replace('_', ' ').title()}")

---
## Export

In [ ]:
def save_floor_plan(grid: np.ndarray, filename: str):
    """Save floor plan to file."""
    if filename.endswith('.npy'):
        np.save(filename, grid)
    elif filename.endswith('.txt'):
        np.savetxt(filename, grid, fmt='%d')
    elif filename.endswith('.png'):
        fig, ax = plt.subplots(figsize=(10, 10))
        cmap = ListedColormap(['#FFFFFF', '#2C3E50'])
        ax.imshow(grid, cmap=cmap, interpolation='nearest')
        ax.axis('off')
        plt.savefig(filename, bbox_inches='tight', pad_inches=0, dpi=150)
        plt.close()
    print(f"Saved to {filename}")

# Uncomment to save:
# save_floor_plan(plan, 'floor_plan.npy')
# save_floor_plan(plan, 'floor_plan.png')